In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')

import definition
from embedding import Preprocessor, Embedding
import pandas as pd
import numpy as np

C:\Tools\Anaconda3\envs\deeplearning\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Data Loading, Preprocessing, and Selection

In [2]:
df = pd.read_csv(definition.DATA_RAW_FILE)
df = df.drop_duplicates('review_id', 'first').reset_index().drop('index', axis=1)
print(len(df))
df.head()

144353


,review_id,review_text,review_source,review_score,category
0,73a965356fe43aa02b113b684f1a365c,Kotor berdebu. Saya tdk berhenti bersin ketika...,befc88fe53f24cb70ce7f9d2efd89858,5.5,kebersihan
1,35f506b353ff95aed86131fb01fb43bc,oke cuma air wastafel berwarna keruh mohon di...,4c5d06b02c97731aaa976179c62dcf76,3.0,fasilitas
2,e93324022b758ce67710708ac115cd18,kamar ada semutnya. kamar mandi bermasalah. bu...,4c5d06b02c97731aaa976179c62dcf76,2.0,kebersihan
3,dff13be030a1a10741848a2d9622cee4,"Kamar mandi bau, airnya bau",4c5d06b02c97731aaa976179c62dcf76,1.0,kebersihan
4,beeff65d3c2e32934261586493934f99,"kamarnya bersih dan nyaman, tetapi perlu di pe...",4c5d06b02c97731aaa976179c62dcf76,3.0,fasilitas


In [3]:
sampling_percentage = 0.6

index = np.arange(len(df))
np.random.shuffle(index)
index_train = index[:int(sampling_percentage * len(df))]

corpus = df['review_text'].values[index_train]
corpus = corpus.astype(str)
corpus[:5]

array(['kamar mandi kurang bersih. tempatnya lumayan\n',
       'pasilitas seperti Tv,canel kabur, kebersihan kurang.',
       'Check in can be earlier.', 'mantaaaap sekali', 'sangat nyaman'],
      dtype='<U2863')

In [4]:
prep_corpus = []
for review in corpus:
    prep_corpus.append(Preprocessor.tokenize(review))
print(len(prep_corpus))
prep_corpus[:5]

86611


[['kamar', 'mandi', 'kurang', 'bersih', '.', 'tempatnya', 'lumayan'],
 ['pasilitas',
  'seperti',
  'tv',
  ',',
  'canel',
  'kabur',
  ',',
  'kebersihan',
  'kurang',
  '.'],
 ['check', 'in', 'can', 'be', 'earlier', '.'],
 ['mantaaaap', 'sekali'],
 ['sangat', 'nyaman']]

## Embedding Construction

In [5]:
embedding = Embedding(min_count=5, size=25, min_n=3, max_n=3, iter=10)
embedding.build_vocab(prep_corpus)
embedding.train(prep_corpus, verbose=True)

Epoch #10/10...


In [6]:
embedding.save(definition.MODEL_EMBEDDING_FASTTEXT)

In [7]:
emb = Embedding()
emb.load(definition.MODEL_EMBEDDING_FASTTEXT)